# Coding: Machine Translation with Transformer

 - Dataset: wmt-17, en-zh, select 1m high-quality pairs of data
 - Model: Seq2seq with Transformer (Attention is all you need)
 - GPU: 1660TI

In [1]:
# 可以先测试网络连接
import requests
try:
    response = requests.get("https://huggingface.co")
    print("网络连接正常")
except:
    print("网络连接可能存在问题")

网络连接正常


In [2]:
# Download the data & select 5m high-quality pairs

from datasets import load_dataset
import re

# load full wmt-17 en-zh dataset
full_dataset = load_dataset("wmt/wmt17", "zh-en", split="train", cache_dir=r"D:\Developer\LLM\FuggingFace-cache-model")

# Length & Ratio filter
def is_high_quality(x):
    import re  # 添加这一行
    en = x["translation"]["en"]
    zh = x["translation"]["zh"]
    if not en or not zh:
        return False
    if len(en) < 3 or len(zh) < 3:
        return False
    if len(en) > 100 or len(zh) > 100:
        return False
    ratio = len(en) / len(zh)
    if ratio < 0.5 or ratio > 2:
        return False
    if not re.search(r'[\u4e00-\u9fff]', zh):
        return False
    return True

filtered_dataset = full_dataset.filter(is_high_quality, num_proc=10)

# 选出最多500万条数据
dataset = filtered_dataset.select(range(min(5_000_000, len(filtered_dataset))))

print("Full Dataset Size: ", len(full_dataset))
print("Filtered Dataset Size: ", len(filtered_dataset))
print("Dataset Size: ", len(dataset))

# print 10 samples
sample = dataset.shuffle(seed=42).select(range(10))
print("-"*100)
for i in sample:
    print(i["translation"]["en"])
    print(i["translation"]["zh"])
    print("-"*100)

Full Dataset Size:  25134743
Filtered Dataset Size:  1141860
Dataset Size:  1141860
----------------------------------------------------------------------------------------------------
Zambia (7)
赞比亚(7)
----------------------------------------------------------------------------------------------------
15:00 to 18:00 Informal consultations (closed) Conference Room 5 (NLB)
下午3:00－6:00 非正式磋商(闭门会议) 第5会议室(北草坪会议大楼)
----------------------------------------------------------------------------------------------------
Spain
西班牙
----------------------------------------------------------------------------------------------------
Mr. Robert Morrison
Robert Morrison先生 加拿大自然资源部
----------------------------------------------------------------------------------------------------
This satisfied the kids, but not the husband.
"孩子们得到了满意的答案, 但她的丈夫却没有。
----------------------------------------------------------------------------------------------------
Shutaro Omura (Japan)
Shutaro Omura（日本）
---------------

In [3]:
# Create PyTorch Dataset and DataLoader for training

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

# 继承Dataset构建英中文翻译数据集
class TranslationDataset(Dataset):
    def __init__(self, hf_dataset, tokenizer_en, tokenizer_zh, max_length=100):
        """
        PyTorch Dataset wrapper for HuggingFace translation dataset

        Args:
            hf_dataset: HuggingFace dataset with translation pairs
            tokenizer_en: English tokenizer (optional, can be added later)
            tokenizer_zh: Chinese tokenizer (optional, can be added later)
            max_length: Maximum sequence length
        """
        self.dataset = hf_dataset
        self.tokenizer_en = tokenizer_en
        self.tokenizer_zh = tokenizer_zh
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        en_text = item["translation"]["en"]
        zh_text = item["translation"]["zh"]

        # 对英文文本进行tokenizer，截取max_length，不够会进行填充，返回pytorch张量
        en_tokens = self.tokenizer_en(en_text,
                                        max_length=self.max_length,
                                        padding='max_length',
                                        truncation=True,
                                        return_tensors='pt')

        # 对中文文本进行tokenizer，截取max_length，不够会进行填充，返回pytorch张量
        zh_tokens = self.tokenizer_zh(zh_text,
                                        max_length=self.max_length,
                                        padding='max_length',
                                        truncation=True,
                                        return_tensors='pt')

        # 返回数据结构
        return {
                'source_ids': en_tokens['input_ids'].squeeze(),     # 去除张量的多余维度，输出以为数字数组，文本对应的tokenizer后的数组
                'target_ids': zh_tokens['input_ids'].squeeze(),
                'source_text': en_text,         # 原始文本
                'target_text': zh_text
        }

# 构建训练和验证数据集
def create_dataloaders(dataset, batch_size=128, num_workers=0, train_split=0.95):
    """
    Create train and validation DataLoaders from HuggingFace dataset

    Args:
        dataset: HuggingFace dataset with translation pairs
        batch_size: Batch size for DataLoaders
        num_workers: Number of worker processes for data loading
        train_split: Fraction of data to use for training

    Returns:
        train_dataloader, val_dataloader, train_dataset, val_dataset
    """

    # Split dataset into train and validation
    train_size = int(train_split * len(dataset))

    # Create indices for splitting
    indices = list(range(len(dataset)))
    train_indices, val_indices = train_test_split(indices,
                                                train_size=train_size,
                                                random_state=42)

    # Create train and validation datasets
    train_dataset_hf = dataset.select(train_indices)
    val_dataset_hf = dataset.select(val_indices)

    # tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    tokenizer_en = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokenizer_zh = AutoTokenizer.from_pretrained("bert-base-chinese")

    # get vocab sizes
    vocab_size_en = tokenizer_en.vocab_size
    vocab_size_zh = tokenizer_zh.vocab_size

    print(f"Vocab size for en: {vocab_size_en}")
    print(f"Vocab size for zh: {vocab_size_zh}")

    # Create PyTorch datasets， 得到tokenizer后的训练数据集和验证数据集
    train_dataset = TranslationDataset(train_dataset_hf, tokenizer_en, tokenizer_zh)
    val_dataset = TranslationDataset(val_dataset_hf, tokenizer_en, tokenizer_zh)

    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")

    # Create DataLoaders 创建了pytorch数据加载器，封装了数据集，并定义了batch_size，shuffle，num_workers，方便训练时设置数据加载相关的超参数
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )

    val_dataloader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )

    print(f"Train DataLoader: {len(train_dataloader)} batches")
    print(f"Validation DataLoader: {len(val_dataloader)} batches")

    return train_dataloader, val_dataloader, vocab_size_en, vocab_size_zh

def test_dataloader(dataloader):
    """Test the DataLoader by printing a sample batch"""
    print("\n" + "="*50)
    print("Sample batch from DataLoader:")
    print("="*50)

    for batch in dataloader:
        print(f"Batch size: {len(batch['source_text'])}")
        print(f"Source example: {batch['source_text'][0]}")
        print(f"Source tokens: {batch['source_ids'][0]}")
        print(f"Target example: {batch['target_text'][0]}")
        print(f"Target tokens: {batch['target_ids'][0]}")
        break

train_dataloader, val_dataloader, encoder_vocab_size, decoder_vocab_size = create_dataloaders(dataset, batch_size=64)
test_dataloader(train_dataloader)
test_dataloader(val_dataloader)

Vocab size for en: 30522
Vocab size for zh: 21128
Train dataset size: 1084767
Validation dataset size: 57093
Train DataLoader: 16950 batches
Validation DataLoader: 893 batches

Sample batch from DataLoader:
Batch size: 64
Source example: Iraq
Source tokens: tensor([ 101, 5712,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
Target example: 伊拉克
Target tokens: tensor([ 101,  823, 2861, 1046,  1

In [ ]:
# Define the Transformer model for Machine Translation
# Following "Attention is All You Need" architecture

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0 # 模型维度必须能被注意力头数整除，模型维度就相当于是每个token在embedding编码后的维度

        self.d_model = d_model  # 模型维度 (int)
        self.num_heads = num_heads  # 注意力头数 (int)
        self.d_k = d_model // num_heads # 每个注意力头维度 (int)

        self.W_q = nn.Linear(d_model, d_model)  # 查询权重矩阵 (d_model × d_model)
        self.W_k = nn.Linear(d_model, d_model)  # 键权重矩阵 (d_model × d_model)
        self.W_v = nn.Linear(d_model, d_model)  # 值权重矩阵 (d_model × d_model)
        self.W_o = nn.Linear(d_model, d_model)  # 输出权重矩阵 (d_model × d_model)

        # dropout 层
        self.dropout = nn.Dropout(dropout)

    # 实现缩放点积注意力机制。
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """
        Calculate attention scores 计算注意力分值
            Q: 查询张量，shape: (batch_size, num_heads, seq_len, d_k)
            K: 键张量，shape: (batch_size, num_heads, seq_len, d_k)
            K.transpose(-2, -1): 转置后的键张量，shape: (batch_size, num_heads, d_k, seq_len)
            torch.matmul(Q, K.transpose(-2, -1)): 矩阵乘法，shape: (batch_size, num_heads, seq_len, seq_len)
            scores: 注意力分数，shape: (batch_size, num_heads, seq_len, seq_len)
        """
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k) # (batch_size, num_heads, seq_len, seq_len)，相当于计算查询和键之间的相似度，当查询和键是同一个矩阵的时候，就是计算两个向量的自我相似度。

        """
        Apply mask if provided
            mask: 遮蔽张量，shape: (batch_size, 1, seq_len, seq_len) 或 (batch_size, num_heads, seq_len, seq_len)
            scores: 遮蔽后的注意力分数，shape: (batch_size, num_heads, seq_len, seq_len)
        """
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # Apply softmax
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        # 注意力权重，shape: (batch_size, num_heads, seq_len, seq_len)

        """
        Apply attention to values
            V: 值张量，shape: (batch_size, num_heads, seq_len, d_k)
            output: 注意力输出，shape: (batch_size, num_heads, seq_len, d_k)
        """
        output = torch.matmul(attention_weights, V)

        return output, attention_weights # 返回注意力输出output:(batch_size, num_heads, seq_len, d_k) 和注意力权重attention_weights: (batch_size, num_heads, seq_len, seq_len)

    """
    query: 查询输入，shape: (batch_size, seq_len, d_model)
    key: 键输入，shape: (batch_size, seq_len, d_model)
    value: 值输入，shape: (batch_size, seq_len, d_model)
    batch_size: 批次大小 (int)
    """
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        """
        Linear transformations and reshape
            self.W_q(query): 线性变换，shape: (batch_size, seq_len, d_model)
            .view(batch_size, -1, self.num_heads, self.d_k): 重塑，shape: (batch_size, seq_len, num_heads, d_k)
            Q, K, V: 转置后，shape: (batch_size, num_heads, seq_len, d_k)
        """
        Q = self.W_q(query).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        """
        Apply attention
            attention_output: 注意力输出，shape: (batch_size, num_heads, seq_len, d_k)
            attention_weights: 注意力权重，shape: (batch_size, num_heads, seq_len, seq_len)
        """
        attention_output, attention_weights = self.scaled_dot_product_attention(Q, K, V, mask)

        """
        Concatenate heads
            attention_output.transpose(1, 2): 转置，shape: (batch_size, seq_len, num_heads, d_k)
            .contiguous().view(batch_size, -1, self.d_model): 连续化并重塑，shape: (batch_size, seq_len, d_model)
        """
        attention_output = attention_output.transpose(1, 2).contiguous().view(
            batch_size, -1, self.d_model)

        # Final linear transformation
        output = self.W_o(attention_output)

        return output   # output: 最终输出，shape: (batch_size, seq_len, d_model)

# 定义位置编码
class PositionalEncoding(nn.Module):
    """
    d_model: 模型维度 (int)
    max_len: 序列最大长度 (int，默认 100)
    """
    def __init__(self, d_model, max_len=100):
        super(PositionalEncoding, self).__init__()

        # pe: 位置编码矩阵，shape: (max_len, d_model), 初始化为零矩阵，用于存储位置编码
        pe = torch.zeros(max_len, d_model)
        '''
        torch.arange(0, max_len, dtype=torch.float): 生成 0 到 max_len-1 的序列，shape: (max_len,)
        .unsqueeze(1): 增加一个维度，shape: (max_len, 1)
        position: 位置索引矩阵，每一行代表一个位置索引
        '''
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # shape: (max_len, 1)
        '''
        torch.arange(0, d_model, 2): 生成 0, 2, 4, ..., d_model-2 的序列（偶数索引）
        .float(): 转换为浮点数
        (-math.log(10000.0) / d_model): 缩放因子
        div_term: 频率分母项，shape: (d_model/2,)
        '''
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        '''
        position * div_term: 广播乘法，shape: (max_len, d_model/2)
        torch.sin(position * div_term): 正弦编码，shape: (max_len, d_model/2)
        pe[:, 0::2]: 选择偶数列（0, 2, 4, ...），shape: (max_len, d_model/2)
        torch.cos(position * div_term): 余弦编码，shape: (max_len, d_model/2)
        pe[:, 1::2]: 选择奇数列（1, 3, 5, ...），shape: (max_len, d_model/2)
        '''
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        '''
        .unsqueeze(0): 增加批次维度，shape: (1, max_len, d_model)
        .transpose(0, 1): 交换第0和第1维度，shape: (max_len, 1, d_model)
        '''
        pe = pe.unsqueeze(0).transpose(0, 1)

        # 将 pe 注册为缓冲区，不会被当作模型参数更新，但在 GPU 上会跟随模型移动
        self.register_buffer('pe', pe) # shape: (max_len, 1, d_model)

    def forward(self, x):
        """
        x: 输入张量，shape: (seq_len, batch_size, d_model)
        self.pe[:x.size(0), :]: 选择与输入序列长度相匹配的位置编码，shape: (seq_len, 1, d_model)
        x + self.pe[:x.size(0), :]: 广播加法，将位置编码添加到输入中，shape: (seq_len, batch_size, d_model)
        返回添加了位置编码的张量
        """
        return x + self.pe[:x.size(0), :]   # shape: (seq_len, batch_size, d_model)